In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import math
from sklearn.model_selection import KFold, train_test_split
from sklearn.datasets import load_iris
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sklearn.linear_model import LogisticRegression


In [2]:
EPOCHS = 50
BATCH_SIZE = 32

In [3]:
def f_get_Normalization(X, norm_mode):
    num_Patient, num_Feature = np.shape(X)

    if norm_mode == 'standard': #zero mean unit variance
        for j in range(num_Feature):
            if np.std(X[:,j]) != 0:
                X[:,j] = (X[:,j] - np.mean(X[:, j]))/np.std(X[:,j])
            else:
                X[:,j] = (X[:,j] - np.mean(X[:, j]))
    elif norm_mode == 'normal': #min-max normalization
        for j in range(num_Feature):
            X[:,j] = (X[:,j] - np.min(X[:,j]))/(np.max(X[:,j]) - np.min(X[:,j]))
    else:
        print("INPUT MODE ERROR!")

    return X


In [4]:
df1 = pd.read_csv('./cleaned_features_final.csv')
df2 = pd.read_csv('./label.csv')

data = np.asarray(df1)
data = f_get_Normalization(data, 'standard')

time = np.asarray(df2[['event_time']])
label = np.asarray(df2[['label']])

num_Category = int(np.max(time)*1.2)
num_Event = int(len(np.unique(label))-1) 

mask = np.zeros([np.shape(time)[0], num_Event, num_Category]) # for the first loss function
for i in range(np.shape(time)[0]):
    if label[i,0] != 0:  #not censored
        mask[i,int(label[i,0]-1),int(time[i,0])] = 1
    else: #label[i,2]==0: censored
        mask[i,:,int(time[i,0]+1):] =  1

In [5]:
x_train, x_test, y_train, y_test = train_test_split(data, df2, test_size=0.2)

x_train = torch.from_numpy(np.asarray(x_train)).float()
y_train = torch.from_numpy(np.asarray(y_train))

x_test = torch.from_numpy(np.asarray(x_test)).float()
y_test = torch.from_numpy(np.asarray(y_test))

In [6]:
class DataSet(Dataset):
  def __init__(self, data, label, mask):
    self.data = data
    self.label = label
    self.mask = mask

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    
    data = self.data[idx]
    label = self.label[idx]
    mask = self.mask[idx]
    return data, label, mask

In [7]:
train_dataset = DataSet(x_train, y_train, mask)
train_loader = DataLoader(train_dataset, batch_size = BATCH_SIZE, shuffle = True, drop_last = False)
test_dataset = DataSet(x_test,y_test, mask)
test_loader = DataLoader(test_dataset, batch_size = BATCH_SIZE, shuffle = False, drop_last = False)

In [8]:
class shared_network(nn.Module):
    def __init__(self):
        super().__init__()
        self.input_dim = 79
        self.h_dim = 200
        self.output_dim = 79 
        self.network = nn.Sequential(
                nn.Linear(self.input_dim,self.h_dim),
                nn.ReLU(),
                nn.Linear(self.h_dim,self.h_dim),
                nn.ReLU(),
                nn.Linear(self.h_dim,self.h_dim),
                nn.ReLU(),    
                nn.Linear(self.h_dim,self.output_dim),
                nn.ReLU(),
            )
       
    def forward(self,x): 
        raw_x = x
        x= self.network(x)
        x = torch.concat([x, raw_x], dim=1)
        return x

In [9]:
shared_network = shared_network()

In [10]:
class sub_network(nn.Module):
    def __init__(self):
        super().__init__()
        self.input_dim = 79*2
        self.h_dim = 200
        self.output_dim = num_Category
        self.network = nn.Sequential(
                nn.Linear(self.input_dim,self.h_dim),
                nn.ReLU(),
                nn.Linear(self.h_dim,self.h_dim),
                nn.ReLU(),
                nn.Linear(self.h_dim,self.h_dim),
                nn.ReLU(),    
                nn.Linear(self.h_dim,self.output_dim),
                nn.ReLU(),               
                )
        
    def forward(self, x): 
        x = self.network(x)
        return x


In [11]:
class Deephit(nn.Module): 
    def __init__(self,shared_network):
        super().__init__()
        self.shared_network = shared_network
        self.num_event = num_Event
        self.num_category = num_Category
        self.h_dim = 200
        self.sub_networks = nn.ModuleList([sub_network() for _ in range(self.num_event)])
        self.nn = nn.Sequential(
            nn.Linear(self.num_event * self.num_category, self.num_event * self.num_category),
            nn.Softmax(dim=1),
            )

    def forward(self, x): 
        shared_output = self.shared_network(x)
        sub_outputs = [sub_net(shared_output) for sub_net in self.sub_networks]
        out = torch.stack(sub_outputs, axis=1)
        out = torch.reshape(out, [-1, self.num_event * self.num_category])
        out = self.nn(out)
        out = torch.reshape(out, [-1, self.num_event, self.num_category])
        return out

In [12]:
def loss_L1(output,label, mask):
        I_1 = torch.sign(label[:,1])
        
        #for uncenosred: log P(T=t,K=k|x)
        tmp1 = torch.sum(torch.sum(mask * output,dim=2), dim=1, keepdim=True)
        tmp1 = I_1 * torch.log(tmp1)

        #for censored: log \sum P(T>t|x)
        tmp2 = torch.sum(torch.sum(mask * output,dim=2), dim=1, keepdim=True)
        tmp2 = (1. - I_1) * torch.log(tmp2)

        return -1* torch.mean(tmp1 + 1.0*tmp2)

In [13]:
deephit = Deephit(shared_network)
optimizer = optim.Adam(deephit.parameters(), lr=1e-4)
 

In [14]:
def train(model ,train_loader, optimizer):
  deephit.train()

  train_loss = 0
  
  for data, label, mask in train_loader:
    data = data
    label = label
    mask = mask
    
    output = deephit(data)
    loss = loss_L1(output, label, mask)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    train_loss += loss.item()
    
  train_loss /= len(train_loader)
  return train_loss

In [15]:
def evaluate(model, test_loader):
  deephit.eval()
  test_loss = 0


  with torch.no_grad():
    for data, label, mask in train_loader:
        data = data
        label = label
        mask = mask

        output = deephit(data)
        test_loss += loss_L1(output, label, mask).item()
    
  test_loss /= len(test_loader)
  return test_loss

In [16]:
result_list = []
for epoch in range(0, EPOCHS + 1):
  train_loss = train(deephit, train_loader, optimizer)
  val_loss = evaluate(deephit, test_loader)
  print(f"[EPOCH: {epoch}], \tTrain Loss: {train_loss:.4f}, \tVal Loss: {val_loss:.4f}")
  result = {
    'EPOCH': epoch,
    'Train Loss': train_loss,
    'Val Loss': val_loss,
    # 'Val Accuracy': val_accuracy
    }
  
  result_list.append(result)
result_df = pd.DataFrame(result_list)


[EPOCH: 0], 	Train Loss: 4.4181, 	Val Loss: 15.9695
[EPOCH: 1], 	Train Loss: 3.8263, 	Val Loss: 13.0124
[EPOCH: 2], 	Train Loss: 3.3541, 	Val Loss: 11.8247
[EPOCH: 3], 	Train Loss: 2.9477, 	Val Loss: 9.3260
[EPOCH: 4], 	Train Loss: 2.2974, 	Val Loss: 6.4974
[EPOCH: 5], 	Train Loss: 1.6517, 	Val Loss: 4.3488
[EPOCH: 6], 	Train Loss: 1.2282, 	Val Loss: 3.2609
[EPOCH: 7], 	Train Loss: 0.9407, 	Val Loss: 2.5226
[EPOCH: 8], 	Train Loss: 0.7723, 	Val Loss: 2.0867
[EPOCH: 9], 	Train Loss: 0.6455, 	Val Loss: 1.7586
[EPOCH: 10], 	Train Loss: 0.5545, 	Val Loss: 1.5410
[EPOCH: 11], 	Train Loss: 0.4863, 	Val Loss: 1.3814
[EPOCH: 12], 	Train Loss: 0.4387, 	Val Loss: 1.2372
[EPOCH: 13], 	Train Loss: 0.4004, 	Val Loss: 1.1228
[EPOCH: 14], 	Train Loss: 0.3702, 	Val Loss: 1.0237
[EPOCH: 15], 	Train Loss: 0.3227, 	Val Loss: 0.9116
[EPOCH: 16], 	Train Loss: 0.2925, 	Val Loss: 0.8651
[EPOCH: 17], 	Train Loss: 0.2780, 	Val Loss: 0.7676
[EPOCH: 18], 	Train Loss: 0.2661, 	Val Loss: 0.7534
[EPOCH: 19], 	Train